<a href="https://colab.research.google.com/github/lengochai97/thesis/blob/master/notebooks/feature_construction/02_Data_Clean_Statistics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Connect to Google Drive

In [0]:
%%capture

import google.colab.drive

google.colab.drive.mount('/content/gdrive', force_remount=True)

# Install Spark and dependencies

In [0]:
import os

os.environ['HADOOP_VERSION'] = '2.7'
os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-8-openjdk-amd64'
os.environ['SPARK_HOME'] = '/opt/spark'
os.environ['SPARK_VERSION'] = '2.4.3'

In [0]:
%%capture

!wget -qN https://archive.apache.org/dist/spark/spark-$SPARK_VERSION/spark-$SPARK_VERSION-bin-hadoop$HADOOP_VERSION.tgz
!tar -xzf spark-$SPARK_VERSION-bin-hadoop$HADOOP_VERSION.tgz -C /opt
!rm spark-$SPARK_VERSION-bin-hadoop$HADOOP_VERSION.tgz
!rm -rf /opt/spark
!ln -s /opt/spark-$SPARK_VERSION-bin-hadoop$HADOOP_VERSION /opt/spark
!pip install -q findspark

# Create SparkSession

In [0]:
import findspark

findspark.init()

In [0]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.master('local[*]').getOrCreate()

# Read files

In [0]:
import json

import pyspark.sql.functions as F
import pyspark.sql.types as T

In [0]:
DATA_PATH = '/content/gdrive/My Drive/dataset/adressa/one_week'

In [0]:
with open(os.path.join(DATA_PATH, 'schema', 'clean.json')) as file:
  clean_schema = T.StructType.fromJson(json.load(file))

In [0]:
df_clean = spark.read.json(os.path.join(DATA_PATH, 'clean'), schema=clean_schema)
df_clean.cache()

DataFrame[userId: string, time: bigint, newsId: string, publishtime: bigint, categoryList: array<string>]

# Do some basic statistics

In [0]:
df_clean.show(truncate=False)

+-------------------------------------------+----------+----------------------------------------+-----------+-----------------------+
|userId                                     |time      |newsId                                  |publishtime|categoryList           |
+-------------------------------------------+----------+----------------------------------------+-----------+-----------------------+
|cx:10aahg3cyumaa128zgcrqm02zi:2gi7mzuwpxq8j|1483386860|cf83d342459ce871e2a8562a91b7dca946e3201a|1483369567 |[nyheter, moreromsdal] |
|cx:10aahg3cyumaa128zgcrqm02zi:2gi7mzuwpxq8j|1483802430|01d923a1af0487ccbf9804bea12f49c12727214a|1483797105 |[100sport, fotball]    |
|cx:10aahg3cyumaa128zgcrqm02zi:2gi7mzuwpxq8j|1483802488|7e98f8a1a50a409a25831be225e01e261dfe04fc|1483790765 |[100sport, vintersport]|
|cx:10aahg3cyumaa128zgcrqm02zi:2gi7mzuwpxq8j|1483826863|7e98f8a1a50a409a25831be225e01e261dfe04fc|1483790765 |[100sport, vintersport]|
|cx:10bpet3dluncp1iz2clzlonsd:27z0v4p30cx3n |1483266354|05e420

## Number of events

In [0]:
n_events = df_clean.count()
n_events

1226167

## Number of users

In [0]:
n_users = df_clean.select(F.column('userId')).distinct().count()
n_users

252196

## Number of items

In [0]:
n_items = df_clean.select(F.column('newsId')).distinct().count()
n_items

1034

## Sparsity 

In [0]:
n_events / (n_users * n_items)

0.0047020894957724236

## Disk usage

In [5]:
!du -sh /content/gdrive/My\ Drive/dataset/adressa/one_week/clean

218M	/content/gdrive/My Drive/dataset/adressa/one_week/clean


## Train and test

In [0]:
train_test_split_time = 1483743600 # 2017/01/06 23:00:00 UTC

### Train size

In [0]:
train_size = df_clean.filter(F.column('time') < train_test_split_time).count()
train_size

1076343

### Test size

In [0]:
test_size = df_clean.filter(F.column('time') >= train_test_split_time).count()
test_size

149824

### Train / test ratio

In [0]:
train_size / test_size

7.184049284493806

## Number of clicks

### Per user

In [0]:
(
    df_clean
    .groupBy('userId')
    .count()
    .agg(
        F.min('count'),
        F.max('count'),
        F.avg('count'),
    )
).show(truncate=False)

+----------+----------+------------------+
|min(count)|max(count)|avg(count)        |
+----------+----------+------------------+
|1         |223       |4.8619605386286855|
+----------+----------+------------------+



### Per item

In [0]:
(
    df_clean
    .groupBy('newsId')
    .count()
    .agg(
        F.min('count'),
        F.max('count'),
        F.avg('count'),
    )
).show(truncate=False)

+----------+----------+-----------------+
|min(count)|max(count)|avg(count)       |
+----------+----------+-----------------+
|1         |43106     |1185.848162475822|
+----------+----------+-----------------+



### Per day

In [0]:
(
    df_clean
    .select(F.dayofyear(F.from_unixtime(F.column('time') + 3600)).alias('day'))
    .filter(
        (F.column('day') > 0) &
        (F.column('day') < 8)
    )
    .groupBy('day')
    .count()
    .agg(
        F.min('count'),
        F.max('count'),
        F.avg('count'),
    )
).show(truncate=False)

+----------+----------+------------------+
|min(count)|max(count)|avg(count)        |
+----------+----------+------------------+
|59856     |233916    |175166.14285714287|
+----------+----------+------------------+



## Number of items published per day

In [0]:
(
    df_clean
    .dropDuplicates(subset=['newsId'])
    .select(F.dayofyear(F.from_unixtime(F.column('publishtime') + 3600)).alias('day'))
    .filter(
        (F.column('day') > 0) &
        (F.column('day') < 8)
    )
    .groupBy('day')
    .count()
    .agg(
        F.min('count'),
        F.max('count'),
        F.avg('count'),
    )
).show(truncate=False)

+----------+----------+----------+
|min(count)|max(count)|avg(count)|
+----------+----------+----------+
|37        |69        |55.0      |
+----------+----------+----------+



## Time between clicks

In [0]:
from pyspark.sql import Window

### Per user

In [0]:
(
    df_clean
    .withColumn(
        'timeLastClick',
        F.sum('time').over(
            Window
            .partitionBy('userId')
            .orderBy('time')
            .rowsBetween(-1, -1)
        )
    )
    .filter(F.column('timeLastClick').isNotNull())
    .withColumn(
        'timeFromLastClick',
        (F.column('time') - F.column('timeLastClick')),
    )
    .agg(
        F.min('timeFromLastClick'),
        F.max('timeFromLastClick'),
        F.avg('timeFromLastClick'),
    )
).show(truncate=False)

+----------------------+----------------------+----------------------+
|min(timeFromLastClick)|max(timeFromLastClick)|avg(timeFromLastClick)|
+----------------------+----------------------+----------------------+
|1                     |603198                |39673.53356516775     |
+----------------------+----------------------+----------------------+



### Per item

In [0]:
(
    df_clean
    .withColumn(
        'timeLastClick',
        F.sum('time').over(
            Window
            .partitionBy('newsId')
            .orderBy('time')
            .rowsBetween(-1, -1)
        )
    )
    .filter(F.column('timeLastClick').isNotNull())
    .withColumn(
        'timeFromLastClick',
        (F.column('time') - F.column('timeLastClick')),
    )
    .agg(
        F.min('timeFromLastClick'),
        F.max('timeFromLastClick'),
        F.avg('timeFromLastClick'),
    )
).show(truncate=False)

+----------------------+----------------------+----------------------+
|min(timeFromLastClick)|max(timeFromLastClick)|avg(timeFromLastClick)|
+----------------------+----------------------+----------------------+
|0                     |594142                |137.89515424039675    |
+----------------------+----------------------+----------------------+

